# Conexión a MongoDB - Llego Backend

In [1]:
# Importar librerías necesarias
from motor.motor_asyncio import AsyncIOMotorClient
import asyncio
from dotenv import load_dotenv
import os

In [2]:
# Cargar variables de entorno desde .env
load_dotenv()

MONGODB_URL = os.getenv('MONGODB_URL')
MONGODB_DATABASE = os.getenv('MONGODB_DATABASE')

print(f"Database: {MONGODB_DATABASE}")

Database: llego


In [3]:
# Crear cliente de MongoDB
client = AsyncIOMotorClient(MONGODB_URL)
db = client[MONGODB_DATABASE]

print(f"Conectado a la base de datos: {MONGODB_DATABASE}")

Conectado a la base de datos: llego


In [ ]:
# Cerrar conexión
client.close()
print("Conexión cerrada")

## Insertar Categorías en MongoDB

In [4]:
# Definir estructura de categorías y subcategorías
from datetime import datetime

BASE_URL = "https://bucket-production-435ad.up.railway.app"

# Estructura de datos
categories_data = [
    {
        "name": "Comida",
        "slug": "comida",
        "subcategories": [
            {"name": "Italiana", "slug": "italiana"},
            {"name": "Platos Fuertes", "slug": "platos_fuertes"},
            {"name": "Vegetariana", "slug": "vegetariana"}
        ]
    },
    {
        "name": "Tienda",
        "slug": "tienda",
        "subcategories": [
            {"name": "Batidos y Cócteles", "slug": "batidos_y_cocteles"},
            {"name": "Bebidas Enlatadas", "slug": "bebidas_enlatadas"},
            {"name": "Botellas", "slug": "botellas"}
        ]
    }
]

# Construir documentos con URLs
categories_to_insert = []
for category in categories_data:
    subcategories = []
    for subcat in category["subcategories"]:
        subcategories.append({
            "name": subcat["name"],
            "imageUrl": f"{BASE_URL}/subcategory/{subcat['slug']}.png"
        })
    
    categories_to_insert.append({
        "name": category["name"],
        "imageUrl": f"{BASE_URL}/category/{category['slug']}.png",
        "subcategories": subcategories,
        "createdAt": datetime.utcnow()
    })

print("Categorías preparadas para insertar:")
for cat in categories_to_insert:
    print(f"\n- {cat['name']}")
    print(f"  URL: {cat['imageUrl']}")
    print(f"  Subcategorías: {len(cat['subcategories'])}")

Categorías preparadas para insertar:

- Comida
  URL: https://bucket-production-435ad.up.railway.app/category/comida.png
  Subcategorías: 3

- Tienda
  URL: https://bucket-production-435ad.up.railway.app/category/tienda.png
  Subcategorías: 3


In [5]:
# Insertar categorías en MongoDB
async def insert_categories():
    # Limpiar colección existente (opcional)
    await db.categories.delete_many({})
    
    # Insertar categorías
    result = await db.categories.insert_many(categories_to_insert)
    
    print(f"\n✅ {len(result.inserted_ids)} categorías insertadas exitosamente!")
    print(f"IDs insertados: {[str(id) for id in result.inserted_ids]}")
    
    return result.inserted_ids

# Ejecutar inserción
inserted_ids = await insert_categories()


✅ 2 categorías insertadas exitosamente!
IDs insertados: ['68e3c8606db8c4398fb9a922', '68e3c8606db8c4398fb9a923']


In [6]:
# Verificar datos insertados
async def verify_categories():
    categories = await db.categories.find().to_list(length=None)
    
    print(f"\n📊 Total de categorías en BD: {len(categories)}\n")
    
    for cat in categories:
        print(f"ID: {cat['_id']}")
        print(f"Nombre: {cat['name']}")
        print(f"Imagen: {cat['imageUrl']}")
        print(f"Subcategorías ({len(cat['subcategories'])}):")
        for subcat in cat['subcategories']:
            print(f"  - {subcat['name']}: {subcat['imageUrl']}")
        print()

# Verificar
await verify_categories()


📊 Total de categorías en BD: 2

ID: 68e3c8606db8c4398fb9a922
Nombre: Comida
Imagen: https://bucket-production-435ad.up.railway.app/category/comida.png
Subcategorías (3):
  - Italiana: https://bucket-production-435ad.up.railway.app/subcategory/italiana.png
  - Platos Fuertes: https://bucket-production-435ad.up.railway.app/subcategory/platos_fuertes.png
  - Vegetariana: https://bucket-production-435ad.up.railway.app/subcategory/vegetariana.png

ID: 68e3c8606db8c4398fb9a923
Nombre: Tienda
Imagen: https://bucket-production-435ad.up.railway.app/category/tienda.png
Subcategorías (3):
  - Batidos y Cócteles: https://bucket-production-435ad.up.railway.app/subcategory/batidos_y_cocteles.png
  - Bebidas Enlatadas: https://bucket-production-435ad.up.railway.app/subcategory/bebidas_enlatadas.png
  - Botellas: https://bucket-production-435ad.up.railway.app/subcategory/botellas.png

